In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "svc", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9234 | AUPR: 0.6655 | Acc: 95.36%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9263 | AUPR: 0.6761 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9284 | AUPR: 0.6827 | Acc: 95.46%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9226 | AUPR: 0.6616 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9256 | AUPR: 0.6722 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9281 | AUPR: 0.6834 | Acc: 95.56%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9284 | AUPR: 0.6827 | Acc: 95.46%
Internal test: AUC: 0.9363 | AUPR: 0.7087 | Acc: 95.33%
External test: AUC: 0.9222 | AUPR: 0.6745 | Acc: 95.19%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC


model(kernel=linear, C=0.01)
Valid:         AUC: 0.9227 | AUPR: 0.6366 | Acc: 95.08%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9231 | AUPR: 0.6328 | Acc: 95.01%
model(kernel=linear, C=1)
Valid:         AUC: 0.9224 | AUPR: 0.6292 | Acc: 94.97%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9231 | AUPR: 0.6328 | Acc: 95.01%
Internal test: AUC: 0.9297 | AUPR: 0.6770 | Acc: 94.79%
External test: AUC: 0.9168 | AUPR: 0.6141 | Acc: 94.68%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7697 | AUPR: 0.4446 | Acc: 94.96%
model(n_neighbors=5)
Valid:         AUC: 0.7920 | AUPR: 0.4938 | Acc: 95.25%
model(n_neighbors=7)
Valid:         AUC: 0.8067 | AUPR: 0.5212 | Acc: 95.25%
best model(n_neighbors=7)
Valid:         AUC: 0.8067 | AUPR: 0.5212 | Acc: 95.25%
Internal test: AUC: 0.8283 | AUPR: 0.5568 | Acc: 95.04%
External test: AUC: 0.8194 | AUPR: 0.5325 | Acc: 95.00%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8830 | AUPR: 0.4436 | Acc: 86.41%
best model()
Valid:         AUC: 0.8830 | AUPR: 0.4436 | Acc: 86.41%
Internal test: AUC: 0.8918 | AUPR: 0.4792 | Acc: 85.13%
External test: AUC: 0.8811 | AUPR: 0.4207 | Acc: 84.18%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9234 | AUPR: 0.6595 | Acc: 95.36%
model(C=10.0)
Valid:         AUC: 0.9231 | AUPR: 0.6587 | Acc: 95.34%
model(C=100.0)
Valid:         AUC: 0.9228 | AUPR: 0.6578 | Acc: 95.33%
best model(C=1.0)
Valid:         AUC: 0.9234 | AUPR: 0.6595 | Acc: 95.36%
Internal test: AUC: 0.9301 | AUPR: 0.6941 | Acc: 95.13%
External test: AUC: 0.9163 | AUPR: 0.6386 | Acc: 94.93%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier


model(criterion=gini)
Valid:         AUC: 0.8753 | AUPR: 0.5644 | Acc: 85.31%
model(criterion=log_loss)
Valid:         AUC: 0.8778 | AUPR: 0.5732 | Acc: 84.83%
model(criterion=entropy)
Valid:         AUC: 0.8778 | AUPR: 0.5732 | Acc: 84.83%
best model(criterion=log_loss)
Valid:         AUC: 0.8778 | AUPR: 0.5732 | Acc: 84.83%
Internal test: AUC: 0.8933 | AUPR: 0.6209 | Acc: 86.45%
External test: AUC: 0.8896 | AUPR: 0.5710 | Acc: 86.03%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9173 | AUPR: 0.6583 | Acc: 89.99%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9181 | AUPR: 0.6615 | Acc: 90.25%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9190 | AUPR: 0.6628 | Acc: 90.11%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9160 | AUPR: 0.6592 | Acc: 90.00%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9178 | AUPR: 0.6626 | Acc: 90.25%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9188 | AUPR: 0.6632 | Acc: 90.16%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9160 | AUPR: 0.6592 | Acc: 90.00%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9178 | AUPR: 0.6626 | Acc: 90.25%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9188 | AUPR: 0.6632 | Acc: 90.16%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9190 | AUPR: 0.6628 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9221 | AUPR: 0.6538 | Acc: 95.24%
model(criterion=squared_error)
Valid:         AUC: 0.9222 | AUPR: 0.6546 | Acc: 95.28%
best model(criterion=squared_error)
Valid:         AUC: 0.9222 | AUPR: 0.6546 | Acc: 95.28%
Internal test: AUC: 0.9321 | AUPR: 0.6850 | Acc: 95.04%
External test: AUC: 0.9217 | AUPR: 0.6414 | Acc: 95.06%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9201 | AUPR: 0.6492 | Acc: 95.29%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9206 | AUPR: 0.6497 | Acc: 95.27%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9207 | AUPR: 0.6514 | Acc: 95.33%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9207 | AUPR: 0.6514 | Acc: 95.33%
Internal test: AUC: 0.9223 | AUPR: 0.6766 | Acc: 94.99%
External test: AUC: 0.9141 | AUPR: 0.6261 | Acc: 94.90%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)